# Oscilaciones forzadas

In [1]:
import numpy as np
from numpy import cos, sin, pi, sqrt, arctan
import plotly.graph_objects as go

# import importlib

from IPython.display import HTML

import sys

# # Lo siguiente es necesario para que funcione en Google Colab
# try:
#   import liboptics as lop
# except:
#   # No descarga los archivos (aunque sí descarga .git)
#   !git clone -n https://github.com/epplugins/UTN.git --depth 1
#   !cd UTN;git checkout HEAD optica/liboptics.py
#   sys.path.insert(0, 'UTN/optica')
#   import liboptics as lop

# %display latex

In [2]:
class oscilador:

    def __init__(self, **params):
        self.m = params.get('m', 0.1)
        self.k = params.get('k', np.nan)
        if not np.isnan(self.k):
            self.w0 = params.get('w0', np.nan)
            if not np.isnan(self.w0):
                print("El valor de w0 no se tuvo en cuenta.")
            self.w0 = sqrt(self.k / self.m)
        else:
            self.w0 = params.get('w0', 2*pi)
            self.k = self.m * self.w0 * self.w0

        self.g = params.get('g', 0)
        self.x0 = params.get('x0', 0.1)
        self.v0 = params.get('v0', 0)

        self.wp = sqrt(self.w0 * self.w0 - self.g * self.g)
        self.wRA = sqrt(self.w0 * self.w0 - 2 * self.g * self.g)

        # Forzado: F_o, w, A estacionario, delta
        if self.g == 0:
            self.F0 = 1
            self.w = params.get('w', 0)
            if self.w > 0:
                print("No hay disipación, el valor de w se forzó a cero.")
            self.w = 0
        else:
            self.F0 = params.get('F0', 1)
            self.w = params.get('w', 0)

        # Calcular A y Aest, d
        # self.A = 
        self.Aest = self.calcular_Aest(self.w)
        self.d = self.calcular_delta()


    def __str__(self):
        #if np.isnan(self.data):
        #    self.text_data = "No images loaded."
        #else:
        #    self.text_data = type(self.data)
        #return f"med Movie object\n\ndata = {self.text_data}\nx0 = {self.x0}\ny0 = {self.y0}\nDimensions = {self.L} x {self.L}"
        out = "Sistema oscilatorio:\n\n"
        out = out + "m = " + str(self.m) + " kg \n"
        out = out + "k = " + str(self.k) + " N/m \n"
        out = out + "w0 = " + str(self.w0) + " 1/s \n"
        out = out + "gamma = " + str(self.g) + " 1/s \n"
        out = out + "w' = " + str(self.wp) + " 1/s \n"
        out = out + "wRA = " + str(self.wRA) + " 1/s \n"
        out = out + "F0 = " + str(self.F0) + " N \n"
        out = out + "w = " + str(self.w) + " 1/s \n"
        out = out + "Aest = " + str(self.Aest * 100) + " cm \n"
        out = out + "delta = " + str(self.d) + " rad \n"
        # return f"Sistema oscilatorio:\n\nNumber of frames = {self.data.shape[0]}\nx0 = {self.x0}\ny0 = {self.y0}\nDimensions = {self.L} x {self.L}"
        return out
    
    def calcular_Aest(self, w):
        """Amplitud en régimen estacionario

        Calcula la amplitud para cualquier valor de w, no solo del w guradado. 
        Conveniente para producir gráficos de amplitud con las características del sistema.

        Parameters
        ----------
        w : float
            Frecuencia externa.
        
        Returns
        -------
        float
        """
        gw2 = (2 * self.g * w) * (2 * self.g * w)
        wow = (self.w0 * self.w0 - w * w) * (self.w0 * self.w0 - w * w)
        Fm = self.F0 / self.m
        return Fm/sqrt(wow+gw2)
    
    def calcular_delta(self):
        """Constante de fase (delta) en régimen estacionario

        Parameters
        ----------

        Returns
        -------
        float
        """
        gw2 = 2 * self.g * self.w
        wow = self.w0 * self.w0 - self.w * self.w
        if wow != 0:
            at = arctan(gw2 / wow)
        else:
            at = pi / 2
        if at < 0:
            at = at + pi
        return at

    def F(self, t):
        """F(t)

        Parameters
        ----------
        t : float
            El tiempo.
        
        Returns
        -------
        float
        """

        return self.F0*cos(self.w*t)

    def xest(self, t):
        """x(t) en régimen estacionario

        Parameters
        ----------
        t : float
            El tiempo.
        
        Returns
        -------
        float
        """

        return self.Aest*cos(self.w*t - self.d)

    def vest(self, t):
        """v(t) en régimen estacionario

        Parameters
        ----------
        t : float
            El tiempo.
        
        Returns
        -------
        float
        """

        return -1*self.w*self.Aest*sin(self.w*t - self.d)




In [3]:
s = oscilador(m = 0.2, k = 80, g = 10, F0 = 2, w = 30)
print(s)

Sistema oscilatorio:

m = 0.2 kg 
k = 80 N/m 
w0 = 20.0 1/s 
gamma = 10 1/s 
w' = 17.320508075688775 1/s 
wRA = 14.142135623730951 1/s 
F0 = 2 N 
w = 30 1/s 
Aest = 1.2803687993289596 cm 
delta = 2.2655346029915995 rad 



In [4]:
fig = go.Figure()
fig.update_layout(showlegend=False, width=1050, height=550)
fig.update_yaxes(automargin=False)
fig.update_xaxes(title_text='w [1/s]', titlefont=dict(size=18))
fig.update_yaxes(title_text='A [cm]', titlefont=dict(size=18), title_standoff = 0)
# fig.update_yaxes(showticklabels=False)
# fig.update_yaxes(range = [-1,1])

ws = np.arange(0,50,0.1)
fig.add_trace(
    go.Scatter(
            mode='lines',
            # name="haz",
            x=ws,
            y=[s.calcular_Aest(w)*100 for w in ws]
        )
)

In [5]:
fig = go.Figure()
fig.update_layout(showlegend=True, width=1050, height=550)
fig.update_yaxes(automargin=False)
fig.update_xaxes(title_text='t [s]', titlefont=dict(size=18))
fig.update_yaxes(title_text='x [cm]', titlefont=dict(size=18), title_standoff = 0)
# fig.update_yaxes(showticklabels=False)
# fig.update_yaxes(range = [-1,1])

ts = np.arange(0,0.6,0.001)
fig.add_trace(
    go.Scatter(
            mode='lines',
            name="x(t) [cm]",
            x=ts,
            y=[s.xest(t)*100 for t in ts]
        )
)
fig.add_trace(
    go.Scatter(
            mode='lines',
            name="F(t) [u.a.]",
            x=ts,
            y=[s.F(t)*100*s.Aest/s.F0 for t in ts]
        )
)

In [6]:
fig = go.Figure()
fig.update_layout(showlegend=True, width=1050, height=550)
fig.update_yaxes(automargin=False)
fig.update_xaxes(title_text='t [s]', titlefont=dict(size=18))
# fig.update_yaxes(title_text='v [cm/s]', titlefont=dict(size=18), title_standoff = 0)
# fig.update_yaxes(showticklabels=False)
# fig.update_yaxes(range = [-1,1])

ts = np.arange(0,0.6,0.001)
fig.add_trace(
    go.Scatter(
            mode='lines',
            name="v(t) [cm/s]",
            x=ts,
            y=[s.vest(t)*100 for t in ts]
        )
)
fig.add_trace(
    go.Scatter(
            mode='lines',
            name="F(t) [u.a.]",
            x=ts,
            y=[s.F(t)*100*s.w*s.Aest/s.F0 + 0 for t in ts]
        )
)

### Resonancia en potencia

In [7]:
sqrt(80/0.2)

20.0

In [9]:
m = 0.2
k = 80
g = 10
F0 = 2
w = sqrt(k/m)  # w = w_o
s = oscilador(m = m, k = k, g = g, F0 = F0, w = w)
print(s)

Sistema oscilatorio:

m = 0.2 kg 
k = 80 N/m 
w0 = 20.0 1/s 
gamma = 10 1/s 
w' = 17.320508075688775 1/s 
wRA = 14.142135623730951 1/s 
F0 = 2 N 
w = 20.0 1/s 
Aest = 2.5 cm 
delta = 1.5707963267948966 rad 



In [11]:
fig = go.Figure()
fig.update_layout(showlegend=True, width=1050, height=550, title_text='Resonancia en potencia: delta = pi/2')
fig.update_yaxes(automargin=False)
fig.update_xaxes(title_text='t [s]', titlefont=dict(size=18))
fig.update_yaxes(title_text='x [cm]', titlefont=dict(size=18), title_standoff = 0)
# fig.update_yaxes(showticklabels=False)
# fig.update_yaxes(range = [-1,1])

ts = np.arange(0,0.6,0.001)
fig.add_trace(
    go.Scatter(
            mode='lines',
            name="x(t) [cm]",
            x=ts,
            y=[s.xest(t)*100 for t in ts]
        )
)
fig.add_trace(
    go.Scatter(
            mode='lines',
            name="F(t) [u.a.]",
            x=ts,
            y=[s.F(t)*100*s.Aest/s.F0 for t in ts]
        )
)

In [16]:
fig = go.Figure()
fig.update_layout(showlegend=True, width=1050, height=550, title_text='Resonancia en potencia: delta = pi/2')
fig.update_yaxes(automargin=False)
fig.update_xaxes(title_text='t [s]', titlefont=dict(size=18))
# fig.update_yaxes(title_text='v [cm/s]', titlefont=dict(size=18), title_standoff = 0)
# fig.update_yaxes(showticklabels=False)
# fig.update_yaxes(range = [-1,1])

ts = np.arange(0,0.6,0.001)
fig.add_trace(
    go.Scatter(
            mode='lines',
            name="v(t) [cm/s]",
            x=ts,
            y=[s.vest(t)*100 for t in ts]
        )
)
fig.add_trace(
    go.Scatter(
            mode='lines',
            name="F(t) [u.a.]",
            x=ts,
            y=[1.5*s.F(t)*100*s.w*s.Aest/s.F0 + 0 for t in ts]
        )
)